GCS提取脚本学习

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2

from IPython.display import display,HTML
%matplotlib inline
plt.style.use('ggplot')

#创建数据库连接
sqluser='postgres'
dbname='mimic'
schema_name='mimiciii'
con=psycopg2.connect(dbname=dbname,user=sqluser)
query_schema='set search_path to '+schema_name+';'

1、

In [2]:
query=query_schema+"""
select l.ICUSTAY_ID
  -- merge the ITEMIDs so that the pivot applies to both metavision/carevue data
  , case
      when l.ITEMID in (723,223900) then 723
      when l.ITEMID in (454,223901) then 454
      when l.ITEMID in (184,220739) then 184
      else l.ITEMID end
    as ITEMID

  -- convert the data into a number, reserving a value of 0 for ET/Trach
  , case
      -- endotrach/vent is assigned a value of 0, later parsed specially
      when l.ITEMID = 723 and l.VALUE = '1.0 ET/Trach' then 0 -- carevue
      when l.ITEMID = 223900 and l.VALUE = 'No Response-ETT' then 0 -- metavision

      else VALUENUM
      end
    as VALUENUM
  , l.CHARTTIME
  from CHARTEVENTS l

  -- get intime for charttime subselection
  inner join icustays b
    on l.icustay_id = b.icustay_id

  -- Isolate the desired GCS variables
  where l.ITEMID in
  (
    -- 198 -- GCS
    -- GCS components, CareVue
    184, 454, 723
    -- GCS components, Metavision
    , 223900, 223901, 220739
  )
  -- Only get data for the first 24 hours
  and l.charttime between b.intime and b.intime + interval '1' day
  -- exclude rows marked as error
  and l.error IS DISTINCT FROM 1
  limit 10
"""
df=pd.read_sql_query(query,con)
df

,icustay_id,itemid,valuenum,charttime
0,294638,184,4.0,2191-03-16 08:00:00
1,228232,184,4.0,2175-05-31 02:00:00
2,273430,184,4.0,2108-08-06 13:00:00
3,229441,184,4.0,2178-04-16 13:00:00
4,220597,184,1.0,2149-11-09 16:00:00
5,225176,184,4.0,2172-10-14 19:00:00
6,217847,184,4.0,2134-09-11 21:30:00
7,263738,184,4.0,2167-01-09 00:00:00
8,220597,184,1.0,2149-11-09 23:00:00
9,257980,184,4.0,2135-05-10 07:00:00


这里需要注意的细节：<br>
1、关于GCS的itemid<br>
CV系统： 
198 asGCS total<br>
这一部分遗漏了<br>
723 as GCSVerbal<br>
454 as GCSMotor<br>
184 as GCSEyes<br>
MV系统：<br>
--    223900 GCS - Verbal Response
--    223901 GCS - Motor Response
--    220739 GCS - Eye Opening<br>
2、为了后续处理方便，作者将CV和MV对应的编码进行统一，方便后续处理<br>
3、在语言反应这一指标，做了特殊处理

In [3]:
query=query_schema+"""
SELECT pvt.ICUSTAY_ID
  , pvt.charttime

  -- Easier names - note we coalesced Metavision and CareVue IDs below
  , max(case when pvt.itemid = 454 then pvt.valuenum else null end) as GCSMotor
  , max(case when pvt.itemid = 723 then pvt.valuenum else null end) as GCSVerbal
  , max(case when pvt.itemid = 184 then pvt.valuenum else null end) as GCSEyes

  -- If verbal was set to 0 in the below select, then this is an intubated patient
  , case
      when max(case when pvt.itemid = 723 then pvt.valuenum else null end) = 0
    then 1
    else 0
    end as EndoTrachFlag

  , ROW_NUMBER ()
          OVER (PARTITION BY pvt.ICUSTAY_ID ORDER BY pvt.charttime ASC) as rn

  FROM  (
  select l.ICUSTAY_ID
  -- merge the ITEMIDs so that the pivot applies to both metavision/carevue data
  , case
      when l.ITEMID in (723,223900) then 723
      when l.ITEMID in (454,223901) then 454
      when l.ITEMID in (184,220739) then 184
      else l.ITEMID end
    as ITEMID

  -- convert the data into a number, reserving a value of 0 for ET/Trach
  , case
      -- endotrach/vent is assigned a value of 0, later parsed specially
      when l.ITEMID = 723 and l.VALUE = '1.0 ET/Trach' then 0 -- carevue
      when l.ITEMID = 223900 and l.VALUE = 'No Response-ETT' then 0 -- metavision

      else VALUENUM
      end
    as VALUENUM
  , l.CHARTTIME
  from CHARTEVENTS l

  -- get intime for charttime subselection
  inner join icustays b
    on l.icustay_id = b.icustay_id

  -- Isolate the desired GCS variables
  where l.ITEMID in
  (
    -- 198 -- GCS
    -- GCS components, CareVue
    184, 454, 723
    -- GCS components, Metavision
    , 223900, 223901, 220739
  )
  -- Only get data for the first 24 hours
  and l.charttime between b.intime and b.intime + interval '1' day
  -- exclude rows marked as error
  and l.error IS DISTINCT FROM 1
  ) pvt
  group by pvt.ICUSTAY_ID, pvt.charttime
  limit 15
"""
df=pd.read_sql_query(query,con)
df

,icustay_id,charttime,gcsmotor,gcsverbal,gcseyes,endotrachflag,rn
0,200001,2181-11-25 19:19:00,6.0,5.0,4.0,0,1
1,200001,2181-11-25 23:49:00,6.0,5.0,4.0,0,2
2,200001,2181-11-26 06:09:00,6.0,5.0,4.0,0,3
3,200001,2181-11-26 09:00:00,6.0,5.0,4.0,0,4
4,200001,2181-11-26 12:00:00,6.0,5.0,3.0,0,5
5,200001,2181-11-26 16:55:00,6.0,5.0,4.0,0,6
6,200003,2199-08-02 20:00:00,6.0,5.0,4.0,0,1
7,200003,2199-08-03 01:00:00,6.0,5.0,4.0,0,2
8,200003,2199-08-03 07:00:00,6.0,5.0,4.0,0,3
9,200003,2199-08-03 11:00:00,6.0,5.0,4.0,0,4
